In [ ]:

#  Demonstrating using Cypher to create bi-directional edges ..
#



#  Setup stuff: Connectivity

In [1]:
#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


In [2]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph_BD"

print("--")


--


In [ ]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [4]:

#  CREATE GRAPH

my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph_BD, 7JHTSZYkey1cR5MSvRSZWngbj6f2fUYPe6mS443m6Bt, 0>


In [5]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph_BD, 7JHTSZYkey1cR5MSvRSZWngbj6f2fUYPe6mS443m6Bt, 0>


# Create: Vertices/nodes, edges ..

In [6]:
import pandas as pd

print("--")

--


In [7]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
   ["LAX", "Los Angeles"             , "Airport"],
   ["SLC", "Salt Lake City"          , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

df_airports.head(20)


,airport_code,airport_name,LABEL
0,MKE,Milwaukee,Airport
1,ORD,Chicago O-Hare,Airport
2,SJC,San Jose,Airport
3,DEN,Denver,Airport
4,LAX,Los Angeles,Airport
5,SLC,Salt Lake City,Airport


In [8]:

#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],
   ["LAX", "ORD", 2029, 1, "FLIES_TO" ],
   ["SLC", "ORD", 1394, 1, "FLIES_TO" ],
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

df_flights.head(20)


,START_ID,END_ID,DISTANCE,NUM_HOPS,TYPE
0,MKE,ORD,66,1,FLIES_TO
1,DEN,ORD,886,1,FLIES_TO
2,SJC,ORD,1829,1,FLIES_TO
3,SJC,ORD,1829,1,FLIES_TO
4,LAX,ORD,2029,1,FLIES_TO
5,SLC,ORD,1394,1,FLIES_TO


In [9]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


/opt/conda/lib/python3.8/site-packages/katana/remote/aio/import_data/dataframe_importer/__init__.py:242: UserWarning: unsupported dtype 'object' for 'airport_code' was coerced to 'string'
  warnings.warn(f"unsupported dtype '{original_dtype}' for '{id_col.name}' was coerced to '{id_col.dtype}'")
/opt/conda/lib/python3.8/site-packages/katana/remote/aio/import_data/dataframe_importer/__init__.py:242: UserWarning: unsupported dtype 'object' for 'START_ID' was coerced to 'string'
  warnings.warn(f"unsupported dtype '{original_dtype}' for '{id_col.name}' was coerced to '{id_col.dtype}'")
/opt/conda/lib/python3.8/site-packages/katana/remote/aio/import_data/dataframe_importer/__init__.py:242: UserWarning: unsupported dtype 'object' for 'END_ID' was coerced to 'string'
  warnings.warn(f"unsupported dtype '{original_dtype}' for '{id_col.name}' was coerced to '{id_col.dtype}'")


          0/? [?op/s]

--


#  Diagnostics:

In [10]:

display(my_graph.num_nodes())
display(my_graph.num_edges())


          0/? [?op/s]

6

          0/? [?op/s]

6

In [15]:

#  What's in the vertices/nodes ..

l_query  = """
   MATCH ( n ) 
   RETURN n
   """.format()

l_result = my_graph.query_unpaginated(l_query)

print(tabulate(l_result, headers="keys", tablefmt="psql", showindex = False))


          0/? [?op/s]

+---------+-------------+-----------+------------------+------------------+--------------+
|   id(n) | labels(n)   | n.LABEL   | n.airport_code   | n.airport_name   | n datatype   |
|---------+-------------+-----------+------------------+------------------+--------------|
|       0 | ['Airport'] | Airport   | MKE              | Milwaukee        | node         |
|       5 | ['Airport'] | Airport   | SLC              | Salt Lake City   | node         |
|       1 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   | node         |
|       2 | ['Airport'] | Airport   | SJC              | San Jose         | node         |
|       3 | ['Airport'] | Airport   | DEN              | Denver           | node         |
|       4 | ['Airport'] | Airport   | LAX              | Los Angeles      | node         |
+---------+-------------+-----------+------------------+------------------+--------------+


In [16]:

#  What's in the edge .. 

l_query  = """
   MATCH (n) - [r] -> (m)
   RETURN r
   // RETURN  n.id AS FROM, m.id AS TO, r.DISTANCE AS DISTANCE, r.NUM_HOPS AS NUM_HOPS
   """.format()

l_result = my_graph.query_unpaginated(l_query)

print(tabulate(l_result, headers = "keys", tablefmt = "psql", showindex = False))


          0/? [?op/s]

+------------------+----------------------+---------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------+
|   id(endNode(r)) | labels(endNode(r))   |   id(r) | type(r)   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |   id(startNode(r)) | labels(startNode(r))   | r datatype   |
|------------------+----------------------+---------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------|
|                1 | ['Airport']          |       3 | FLIES_TO  |          886 |            1 | FLIES_TO |                  3 | ['Airport']            | edge         |
|                1 | ['Airport']          |       4 | FLIES_TO  |         2029 |            1 | FLIES_TO |                  4 | ['Airport']            | edge         |
|                1 | ['Airport']          |       1 | FLIES_TO  |         1829 |            1 | FLIES_TO |                  2 | ['Airport']            | edge   

In [ ]:

#  Use Cypher to create reverse edges ..
#

l_query  = """
   MATCH (n) - [ r ] -> (m)
   RETURN n, r, m
   """.format()

l_return = my_graph.query(l_query, contextualize=True)



